In [ ]:
#"Hard" algorithm
#https://www.cs.princeton.edu/~chazelle/pubs/mstapprox.pdf 
#"Easy" algorithm
#http://www.wisdom.weizmann.ac.il/~oded/PTW/sublin.pdf

In [ ]:
import networkx as nx
import random

n     = 10
max_w = 5
c     = 1
eps   = 0.05

#G = nx.generators.random_graphs.random_regular_graph(3, n)

generator = nx.generators.cycle_graph
#generator = nx.generators.complete_graph

#n = n*c
#s.random_internet_as_graph(n)
G = generator(n)

for _ in range(c - 1):
    G_ = generator(n)
    G = nx.disjoint_union(G, G_)

n = n * c

for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1, max_w)

mst_gt = nx.algorithms.tree.mst.minimum_spanning_tree(G)

mst_gt_w = 0
#print(mst_gt.edges)
print(list(mst_gt.edges)[:10])
for (u,v) in mst_gt.edges:
    #print((u, v))
    mst_gt_w += G.edges[u,v]['weight']

print(mst_gt_w)

import networkx as nx
import random

n     = 100
max_w = 20
c     = 5
eps   = 0.05

#G = nx.generators.random_graphs.random_regular_graph(3, n)

generator = nx.generators.cycle_graph
#generator = nx.generators.complete_graph

#n = n*c
#s.random_internet_as_graph(n)
G = generator(n)

for _ in range(c - 1):
    G_ = generator(n)
    G = nx.disjoint_union(G, G_)

n = n * c

for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1, max_w)

mst_gt = nx.algorithms.tree.mst.minimum_spanning_tree(G)

mst_gt_w = 0
#print(mst_gt.edges)
#print(list(mst_gt.edges)[:10])
for (u,v) in mst_gt.edges:
    #print((u, v))
    mst_gt_w += G.edges[u,v]['weight']

print(mst_gt_w)

In [ ]:
#Easy version, 31.9 p

import random
import sys

DEBUG = 0

RANDOM_SAMPLE_FACTOR = 200
SOME_BIG_CONSTANT    = 1000

#Vertex to [(neighbor, edge weight), ... ]
memos = {}

def query_node_local(n):
    nbors = G.neighbors(n)
    return [(n2, G.edges[n, n2]["weight"]) for n2 in nbors]

def query_node(n):

    if DEBUG:
        return query_node_local(n)

    if n in memos:
        return memos[n]

    else:
        print(n)
        sys.stdout.flush()
        inp = input().strip("\n").split()[1:]
        memos[n] = []
        if len(inp) < 2:
            return memos[n]

        for i in range(0, len(inp), 2):
            #print("i", i, "len inp", len(inp), "inp", inp)
            memos[n].append((int(inp[i]), int(inp[i + 1])))
        
        return memos[n]

def approx_avg_degree(eps, C, n):
    import random
    n_ = 50 + int(C/eps)
    n = min(n_, 1+ n//1000)
    max_deg = 0

    for i in range(n_):
        node = random.randint(0, n - 1)
        out = query_node(node)

        deg = len(out)
        max_deg = max(max_deg, deg)


    return max_deg


def approx_cc_simple(n, gi, eps, max_w, d_bar):
    import random
    import sys

    r = 50 + int(1/eps * RANDOM_SAMPLE_FACTOR)
    r = min(1 + n//1000, r)

    #Does not check dupes
    vs = [random.randint(0, n - 1) for _ in range(r)]

    betas = [1 for _ in range(int(r))]
    
    for i, u in enumerate(vs):
        X = int( 1/random.random())

        betas[i] = 0
        visited_nodes = set()
        visited_nodes.add(u)

        u_nbors = [v for v in query_node(u) if v[0] != u and v[1] <= gi]

        for v in u_nbors:
            for _ in range(u_nbors.count(v) - 1):
                u_nbors.remove(v)

        if DEBUG:
            for v in u_nbors:
                assert(u_nbors.count(v) == 1)

        if u_nbors == []:
            betas[i] = 1
            continue

        j = 0
        for v in u_nbors:
            j += 1
            visited_nodes.add(v[0])
            v_nbors = query_node(v[0])
            added = False
            for t in v_nbors:
                if t[0] not in visited_nodes and t[0] not in u_nbors and t[1] <= gi:
                    u_nbors.append(t)
                    added = True

            if j > X  or len(visited_nodes) > X :
                betas[i] = 0
                break
            
            if u_nbors[j:] == [] or not added:
                betas[i] = 1
                break


    return n / r * sum(betas)


def approx_mst(n, eps, max_w):

    d_bar = 0
    
    c_bars = []
    for i in range(1, max_w):

        cc_bar = approx_cc_simple(n, i , eps, 4*max_w/eps, d_bar) i
        c_bars.append(cc_bar)

    all_ = approx_cc_simple(n, max_w, 0.05, 4*max_w/eps, d_bar)
    return n - max_w + sum(c_bars) - all_*max_w 

if not DEBUG:
    n = int(input())
    eps = float(input())
    max_w = int(input())

mst = approx_mst(n, eps, max_w)

print("end " + str(mst))
sys.stdout.flush()

if DEBUG == 1:
    print("gt end", mst_gt_w)



In [ ]:
#Hard version
import random
import sys

DEBUG = 1

RANDOM_SAMPLE_FACTOR = 200
SOME_BIG_CONSTANT    = 10

#Vertex to [(neighbor, edge weight), ... ]
memos = {}

def query_node_local(n):
    nbors = G.neighbors(n)
    return [(n2, G.edges[n, n2]["weight"]) for n2 in nbors]

def query_node(n):

    if DEBUG:
        return query_node_local(n)

    if n in memos:
        return memos[n]

    else:
        print(n)
        sys.stdout.flush()
        inp = input().strip("\n").split()[1:]
        memos[n] = []
        if len(inp) < 2:
            return memos[n]

        for i in range(0, len(inp), 2):
            #print("i", i, "len inp", len(inp), "inp", inp)
            memos[n].append((int(inp[i]), int(inp[i + 1])))
        
        return memos[n]

def approx_avg_degree(eps, C, n):
    import random
    n_ = 50 + int(C/eps)
    n = min(n_, 1+ n//1000)
    max_deg = 0

    for i in range(n_):
        node = random.randint(0, n - 1)
        out = query_node(node)

        deg = len(out)
        max_deg = max(max_deg, deg)


    return max_deg


def approx_cc(n, gi, eps, max_w, d_bar):
    import random
    import sys

    r = int(1/eps**2) * RANDOM_SAMPLE_FACTOR


    #Does not check dupes
    vs = [random.randint(0, n - 1) for _ in range(r)]
    #Remove dupes
    #for v in vs:
    #    for _ in range(vs.count(v) - 1):
    #        vs.remove(v)
    

    r = len(vs)
    betas = [0 for _ in range(int(r))]


    for i, u in enumerate(vs):

        # Start a BFS that exits probabilistically 

        seen_edges      = set()
        seen_nodes      = set()
        visited_nodes   = set()
        visited_edges   = set()
        n_seen_edges    = 0      #Seen and visited are the same
        n_seen_nodes    = 0
        n_visited_nodes = 0
        n_visited_edges = 0
        max_degree_seen = 0

        u_neighs = query_node(u)
        u_neighs = [t for t in u_neighs if t[1] <= gi]

        singleton = False

        if len(u_neighs) == 0:
            singleton = True
            betas[i] = 2
            continue

        throws = 0

        dui = len(u_neighs)
        walk_limit = len(u_neighs)
        
        max_degree_seen = len(u_neighs)

        edges_by_depth = [[((u, v[0]), v[1])  for v in u_neighs]]

        for j, u_edges in enumerate(edges_by_depth):
            #edge :: tuple (tuple (int u, int v), float weight)

            #Take a step
            edges_by_depth.append([])
            #V is an edge
            if DEBUG == 1:
                print("start depth", j)
            for v in u_edges:
                if v[0][1] not in visited_nodes:
                    visited_nodes.add(v[0][1])
                    n_visited_nodes += 1
                    visited_edges.add(v[0][1])
                    n_visited_edges += 1

                ts = query_node(v[0][1])
                max_degree_seen = max(max_degree_seen, len(ts))

                #ts is a list of tuples (vertex, weight)
                for t in ts:
                    if ((v[0][1], t[0]), t[1]) not in seen_edges:
                        seen_edges.add(((v[0][1], t[0]), t[1]))
                        n_seen_edges += 1
                    if t[0] not in visited_nodes:
                        visited_nodes.add(t[0])
                        n_visited_nodes += 1
                        edges_by_depth[-1].append(((v[0][0], t[0]), t[1]))

            if max_degree_seen > d_bar or len(visited_nodes) > max_w:
                break

            if len(seen_edges) < walk_limit:
                toss = random.randint(0, 1)
                throws += 1
                if toss == 1:
                    walk_limit = walk_limit*2
                else:
                    break
                #max_degree_seen = max(max_degree_seen, dui)


        if len(u_neighs[j:]) == 0:
            #Finished bfs
            betas[i] = dui * 2**(throws + 1)
   
    return n / (2 * r) * sum(betas)



def approx_mst(n, eps, max_w):

    d_bar = approx_avg_degree(eps, SOME_BIG_CONSTANT, n)
    #d_bar = 0
    
    c_bars = []
    for i in range(1, max_w):

        cc_bar = approx_cc(n, i , eps, 4*max_w/eps, d_bar)   
        c_bars.append(cc_bar)

    all_ = approx_cc_simple(n, max_w, 0.0005, 4*max_w/eps, d_bar)
    if DEBUG == 1:
        print("gt c", c)
        print("est c", all_)

    return n - max_w + sum(c_bars) - all_*max_w 

if not DEBUG:
    n = int(input())
    eps = float(input())
    max_w = int(input())

assert (max_w > 1)

mst = approx_mst(n, eps, max_w)

#assert(mst >= n/2)

print("end " + str(mst))
sys.stdout.flush()

if DEBUG == 1:
    print("gt end", mst_gt_w)

